In [1]:
import numpy as np
import pandas as pd
import umap
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA

D:\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载数据
file_path = "预测.xlsx"
df = pd.read_excel(file_path)

In [3]:
# 加载文本特征
text_embeddings = np.load('PubMedBERT.npy')

In [4]:
# scaler = StandardScaler()
# text_embeddings_scaled = scaler.fit_transform(text_embeddings)

# pca = PCA(n_components=1)
# text_features_pca = pca.fit_transform(text_embeddings_scaled)
text_features_pca = umap.UMAP(n_components=2, random_state=42).fit_transform(MinMaxScaler().fit_transform(text_embeddings))

# 再截取前627行降维结果
#text_features_pca = text_features_pca[627:]

# 动态生成列名
n_pca_features = text_features_pca.shape[1]
pca_columns = [f'Text_Embedding_Combined{i+1}' for i in range(n_pca_features)]

# 转换为DataFrame
text_feature_pca_df = pd.DataFrame(text_features_pca, columns=pca_columns)

D:\anaconda3\envs\tensorflow\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [5]:
X = pd.concat([df, text_feature_pca_df], axis=1)

In [6]:
X

,LN转移个数,腋窝淋巴结状态,PR,HER2+FISH,ki-67,手术前怀孕,治疗后怀孕,治疗后生产,目前月经情况,手术方式,放疗,化疗期间是否应用诺雷德,靶向治疗（赫赛汀或赫赛汀+帕捷特）,化疗方案,内分泌治疗方案,标签,Text_Embedding_Combined1,Text_Embedding_Combined2
0,5,1,10,1,0.20,2,0,0,6,2,1,0,1,2,1,1,14.980383,5.013096
1,2,1,95,2,0.10,1,0,0,1,2,1,0,0,4,1,0,12.068896,8.871240
2,0,0,70,1,0.05,3,0,0,5,1,1,0,0,8,1,0,16.276093,8.185767
3,6,1,3,0,0.60,2,0,0,7,5,1,0,0,4,1,1,16.475365,6.555526
4,0,0,90,3,0.05,4,0,0,6,5,0,0,1,2,1,0,12.591355,5.566292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,0,0,90,0,0.40,2,0,0,2,1,1,0,0,0,1,0,13.169783,8.524548
1362,1,1,95,0,0.45,0,0,0,1,1,1,0,0,0,3,1,12.164911,5.536772
1363,8,1,0,1,0.40,1,0,0,1,4,1,0,0,7,0,1,17.065783,6.023079
1364,0,0,95,2,0.15,3,0,0,2,1,1,0,0,8,0,0,14.051453,7.935132


In [7]:
X.to_excel('预测+语义增强.xlsx', index=False)

# 诊疗手段扩展全

In [1]:
import pandas as pd
from itertools import product

# 读取数据
patient_df = pd.read_excel("诊疗全.xlsx")

# 定义诊疗手段的所有可能取值
手术方式 = [1, 2, 3, 4, 5, 6]
放疗 = [0, 1]
诺雷德 = [0, 1]
靶向治疗 = [0, 1]
化疗方案 = list(range(10))  # 0~9
内分泌治疗方案 = list(range(5))  # 0~4

# 所有组合
all_combinations = list(product(手术方式, 放疗, 诺雷德, 靶向治疗, 化疗方案, 内分泌治疗方案))

# 构建新的数据表
new_data = []

for _, row in patient_df.iterrows():
    base_features = row.to_dict()
    for combo in all_combinations:
        new_row = base_features.copy()
        new_row.update({
            '手术方式': combo[0],
            '放疗': combo[1],
            '化疗期间是否应用诺雷德': combo[2],
            '靶向治疗（赫赛汀或赫赛汀+帕捷特）': combo[3],
            '化疗方案': combo[4],
            '内分泌治疗方案': combo[5]
        })
        new_data.append(new_row)

# 转换为 DataFrame
expanded_df = pd.DataFrame(new_data)

# 保存到新数据表
expanded_df.to_csv('诊疗全_扩展.csv', index=False, encoding='utf-8-sig')